In [ ]:
from src.svg import *
from src.extraction import *
from src.database import *
from src.labels import LABELS
from src.vision import *
from src.matching import *

# Evaluation

In [ ]:
IMAGE_DIRECTORY = os.sep.join(['assets', 'svg'])

def load_first_svg_files(ext='svg'):
    """
    Returns an iterator that yields the image file paths and their labels.
    """    
    files = []
    
    dirs = next(os.walk(IMAGE_DIRECTORY))[1]
    for d in dirs:
        dir_path = os.path.join(IMAGE_DIRECTORY, d)
        
        for i in range(5):
            file = os.listdir(dir_path)[i]
            file = os.path.join(dir_path, file)

            files.append((file, d))
                
    return files

def extract_graphs(images, step=20):
    """
    Extract all graphs from given images.
    """
    i = 0
    graphs = []
    for img, l in images:
        cp = map(to_control_points,img['paths'])
        
        try:
            graphs.append(extract_graph(cp, l, step=step, check_area=False))
        except StopIteration:
            print('drawing was too small')
        
        print(i)
        i += 1
        
    return graphs

In [ ]:
paths = load_first_svg_files()
print(len(paths))

images = load_svg_images(paths)

graphs = extract_graphs(images, step=20)
print(len(graphs))

db = open_database()

In [ ]:
dists = []

for i, query in enumerate(graphs):
    key = descriptor(query, N=DESCRIPTOR_SIZE)
    print(key)

    dist = query_database(db, query)

    dists.append(dist)

    i = np.argmax(dist)
    print(LABELS[i])

    # # Figure Size
    # fig = plt.figure()

    # plt.suptitle(f"file: {paths[i]}\nargmax: {LABELS[i]}")
    
    # # Horizontal Bar Plot
    # plt.bar(range(len(LABELS)), dist)
    # plt.xticks(range(0, len(LABELS), len(LABELS)//10))

    # plt.show()

close_database(db)